In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
def get_dem_data(url, *, row_step=1, col_step=1):
    import requests
    from osgeo import gdal

    r = requests.get(url)
    vsipath = '/vsimem/dem'
    gdal.FileFromMemBuffer(vsipath, r.content)

    return gdal.Open(vsipath).ReadAsArray()[::row_step, ::col_step]

def dataset_array_to_dataframe(dataset_array, *, shift=False, scale=False, max_height=1):
    df = pd.DataFrame.from_records([
        (j, i, a)
        for i, r in enumerate(dataset_array)
        for j, a in enumerate(r)
    ], columns=["x", "y", "h"])

    if shift:
        df.h = df.h + abs(df.h.min())

    if scale:
        a = max_height / (df.h.max() - df.h.min())
        b = -a * df.h.min()
        df.h = a * df.h + b

    return df

In [4]:
# Original: https://blog.datawrapper.de/weekly-ridgeline-plot/

pulsar_path = "https://gist.githubusercontent.com/borgar/31c1e476b8e92a11d7e9/raw/0fae97dab6830ecee185a63c1cee0008f6778ff6/pulsar.csv"

pulsar_df = dataset_array_to_dataframe(pd.read_csv(pulsar_path, header=None).to_numpy(), shift=True)

ggplot(pulsar_df) + \
    geom_area_ridges(aes("x", "y", height="h"), \
                     stat='identity', scale=.25, \
                     color="white", fill="black",
                     sampling=sampling_pick(pulsar_df.shape[0])) + \
    scale_y_continuous(trans='reverse') + \
    ggsize(800, 800) + \
    ggtitle("Post-punk remake") + \
    theme_minimal() + flavor_high_contrast_dark()

In [5]:
# Original: https://nbviewer.org/github/royalosyin/Work-with-DEM-data-using-Python-from-Simple-to-Complicated/blob/master/Sup03-Ridgelines%20Map%20of%20DEM.ipynb

nz_df = dataset_array_to_dataframe(pd.read_csv("data/new_zealand-dem.csv", header=None).to_numpy())

ggplot(nz_df) + \
    geom_area_ridges(aes("x", "y", height="h"), \
                     stat='identity', scale=.002, \
                     color="#08519c", fill="#bdd7e7",
                     sampling=sampling_pick(nz_df.shape[0])) + \
    scale_y_continuous(trans='reverse') + \
    ggsize(800, 800) + \
    ggtitle("New Zealand remake") + \
    theme_minimal()

In [6]:
ivindo_path = "https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/ivindo_river.tif"
max_height = 3.0

ivindo_data = get_dem_data(ivindo_path, row_step=24, col_step=6)
ivindo_df = dataset_array_to_dataframe(ivindo_data, shift=True, scale=True, max_height=max_height)

ggplot(ivindo_df) + \
    geom_area_ridges(aes("x", "y", height="h"), \
                     stat='identity', \
                     size=0, fill="#016450", alpha=1.0/max_height,
                     sampling=sampling_pick(ivindo_df.shape[0])) + \
    scale_y_continuous(trans='reverse') + \
    ggsize(800, 800) + \
    ggtitle("Ivindo river") + \
    theme_minimal()